In [ ]:
# pip install nltk

In [2]:
import pandas as pd
import numpy
import sys
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
nltk.download('stopwords')
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [3]:
file = pd.read_csv("Resources/positive_only.csv")
file

,Unnamed: 0,Review_ID,Rating,Reviewer_Location,Review_Text,Branch,Year,Month,is_good_review,review_clean,neg,neu,pos,compound,characters,words,doc2vec_vector_0,doc2vec_vector_1,doc2vec_vector_2,doc2vec_vector_3,doc2vec_vector_4,word_ability,word_able,word_absolute,word_absolutely,word_accept,word_access,word_accessible,word_accommodate,word_accommodation,word_accordingly,word_account,word_accurate,word_across,word_act,word_action,word_activity,word_actor,word_actual,word_actually,...,word_wise,word_wish,word_with,word_within,word_without,word_woman,word_wonder,word_wonderful,word_wonderland,word_wont,word_woody,word_word,word_work,word_worker,word_world,word_worried,word_worry,word_worse,word_worst,word_worth,word_worthwhile,word_would,word_wow,word_wrap,word_write,word_wrong,word_yeah,word_year,word_yes,word_yesterday,word_yet,word_york,word_you,word_young,word_yr,word_yrs,word_yum,word_yummy,word_zero,word_zone
0,0,122944085,5,United States,Spotlessly clean and so organized considering ...,Disneyland_California,2011,10,1,spotlessly clean organize consider thousand vi...,0.000,0.786,0.214,0.4574,81,12,-0.039490,-0.044649,-0.071815,0.044624,-0.001575,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
1,1,290851203,5,United Kingdom,"Pricey but worth it, unfortunately the big thu...",Disneyland_Paris,2015,7,1,pricey worth unfortunately big thunder mountai...,0.115,0.567,0.317,0.9809,468,87,0.292319,-0.415854,-0.176826,-0.038325,-0.007365,0.0,0.0,0.0,0.150544,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.102765,0.0,0.0,0.0,0.0,0.102825,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
2,2,168087311,4,Canada,"Disneyland is a magical place. Everyone, parti...",Disneyland_California,2013,7,1,disneyland magical place everyone particularly...,0.000,0.810,0.190,0.9214,360,58,0.201055,-0.327959,-0.130096,-0.123862,-0.251303,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
3,3,282417360,5,Pakistan,Disney land is a must when visiting Paris. The...,Disneyland_Paris,2015,6,1,disney land must visit paris frozen show best ...,0.028,0.843,0.129,0.7050,297,58,0.089066,-0.103461,-0.021174,-0.210190,-0.182371,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.139952,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
4,5,633178209,5,United States,Went with a group of Disneyfied adults on a we...,Disneyland_HongKong,2018,10,1,go group disneyfied adult week day park deck h...,0.048,0.742,0.209,0.8122,211,44,-0.088582,-0.066877,-0.123172,-0.041799,-0.188985,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.243605,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3213,3998,220207400,5,Thailand,"In order to stay for the fireworks, reach disn...",Disneyland_HongKong,2014,7,1,order stay fireworks reach disneyland onwards ...,0.000,0.848,0.1

In [4]:
text = file["Review_Text"]
text

0       Spotlessly clean and so organized considering ...
1       Pricey but worth it, unfortunately the big thu...
2       Disneyland is a magical place. Everyone, parti...
3       Disney land is a must when visiting Paris. The...
4       Went with a group of Disneyfied adults on a we...
                              ...                        
3213    In order to stay for the fireworks, reach disn...
3214    There's a reason why they call Disneyland  The...
3215    What a great day we had we did so much before ...
3216    It has been a childhood dream of mine to go to...
3217    Well my wife and I visted in October 2012 for ...
Name: Review_Text, Length: 3218, dtype: object

In [5]:
corpus = text.to_string()
corpus

"0       Spotlessly clean and so organized considering ...\n1       Pricey but worth it, unfortunately the big thu...\n2       Disneyland is a magical place. Everyone, parti...\n3       Disney land is a must when visiting Paris. The...\n4       Went with a group of Disneyfied adults on a we...\n5       love it as always.  wish is was not so crowded...\n6       Having only visited Disneyland Paris previousl...\n7       As a family with two children aged 7 and 8 we ...\n8       What else can you say about Disneyland, it is ...\n9       Those who love Disney will fall in love with D...\n10      First time to go to this place. The place was ...\n11      Pretty great place, been to a lot of Disney pa...\n12      Gosh, I've been going to Disneyland for 30  ye...\n13      We thought now trip to pairs would be complete...\n14      We arrived from Hong Kong Island via MTR from ...\n15      So first of all, we really LOVE Disney.We've b...\n16      Being a big Disneyworld fan, I had low expecta.

In [6]:
def tokenize_words(input):
    # lowercase everything to standardize it
    input = input.lower()

    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(input)

    # if the created token isn't in the stop words, make it part of "filtered"
    # filtered = filter(lambda token: token not in stopwords.words('english'), tokens)
    return " ".join(tokens)

In [7]:
input = tokenize_words(corpus)
input

'0 spotlessly clean and so organized considering 1 pricey but worth it unfortunately the big thu 2 disneyland is a magical place everyone parti 3 disney land is a must when visiting paris the 4 went with a group of disneyfied adults on a we 5 love it as always wish is was not so crowded 6 having only visited disneyland paris previousl 7 as a family with two children aged 7 and 8 we 8 what else can you say about disneyland it is 9 those who love disney will fall in love with d 10 first time to go to this place the place was 11 pretty great place been to a lot of disney pa 12 gosh i ve been going to disneyland for 30 ye 13 we thought now trip to pairs would be complete 14 we arrived from hong kong island via mtr from 15 so first of all we really love disney we ve b 16 being a big disneyworld fan i had low expecta 17 this is our 2nd time here and we ve had such a 18 i am a magic kingdom connoisseur so it s hard 19 this is our second year doing the disney famil 20 we spend 5 days at disney

In [8]:
chars = sorted(list(set(input)))
char_to_num = dict((c, i) for i, c in enumerate(chars))

In [9]:
input_len = len(input)
vocab_len = len(chars)
print ("Total number of characters:", input_len)
print ("Total vocab:", vocab_len)

Total number of characters: 162879
Total vocab: 37


In [10]:
seq_length = 5
x_data = []
y_data = []

In [11]:
for i in range(0, input_len - seq_length, 1):
    in_seq =input[i:i + seq_length]
    out_seq = input[i + seq_length]
    
    x_data.append([char_to_num[char] for char in in_seq])
    y_data.append(char_to_num[out_seq])

In [12]:
n_patterns = len(x_data)
print ("Total Patterns:", n_patterns)

Total Patterns: 162874


In [13]:
X = numpy.reshape(x_data, (n_patterns, seq_length, 1))
X = X/float(vocab_len)
X

array([[[0.02702703],
        [0.        ],
        [0.78378378],
        [0.7027027 ],
        [0.67567568]],

       [[0.        ],
        [0.78378378],
        [0.7027027 ],
        [0.67567568],
        [0.81081081]],

       [[0.78378378],
        [0.7027027 ],
        [0.67567568],
        [0.81081081],
        [0.59459459]],

       ...,

       [[0.08108108],
        [0.02702703],
        [0.05405405],
        [0.08108108],
        [0.        ]],

       [[0.02702703],
        [0.05405405],
        [0.08108108],
        [0.        ],
        [0.43243243]],

       [[0.05405405],
        [0.08108108],
        [0.        ],
        [0.43243243],
        [0.67567568]]])

In [14]:
y = np_utils.to_categorical(y_data)
y

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [30]:
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(128))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))

In [31]:
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [32]:
filepath = "Resources/positive_model_weights_final.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
desired_callbacks = [checkpoint]

In [33]:
model.fit(X, y, epochs=120, batch_size=400, callbacks=desired_callbacks)

Epoch 1/120
408/408 [==============================] - 7s 8ms/step - loss: 3.0841

Epoch 00001: loss improved from inf to 2.93671, saving model to Resources/positive_model_weights_final.hdf5
Epoch 2/120
408/408 [==============================] - 3s 8ms/step - loss: 2.5368

Epoch 00002: loss improved from 2.93671 to 2.45197, saving model to Resources/positive_model_weights_final.hdf5
Epoch 3/120
408/408 [==============================] - 3s 8ms/step - loss: 2.2557

Epoch 00003: loss improved from 2.45197 to 2.21452, saving model to Resources/positive_model_weights_final.hdf5
Epoch 4/120
408/408 [==============================] - 3s 8ms/step - loss: 2.1110

Epoch 00004: loss improved from 2.21452 to 2.08462, saving model to Resources/positive_model_weights_final.hdf5
Epoch 5/120
408/408 [==============================] - 3s 8ms/step - loss: 2.0091

Epoch 00005: loss improved from 2.08462 to 1.99381, saving model to Resources/positive_model_weights_final.hdf5
Epoch 6/120
408/408 [========

In [34]:
filename = "Resources/positive_model_weights_final.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [35]:
num_to_char = dict((i, c) for i, c in enumerate(chars))

In [38]:
start = numpy.random.randint(0, len(x_data) - 1)
pattern = x_data[start]
print("Random Seed:")
print("\"", ''.join([num_to_char[value] for value in pattern]), "\"")

Random Seed:
" ralia "
